In [1]:
"""
quickhull.ipynb

Geometría Computacional - IMAT
ICAI, Universidad Pontificia Comillas

Integrantes del grupo:
    - María González Gómez
    - Lydia Ruiz Martínez
    - David Tarrasa Puebla
"""

def maxX(P):
    """
    Función que calcula el punto con el mayor valor en el eje X de un conjunto de puntos P.
    """
    return max(P)

def minX(P):
    """
    Función que calcula el punto con el menor valor en el eje X de un conjunto de puntos P.
    """
    return min(P)

def orientacionPuntoRecta(p1, p2, p):
    """
    Función que calcula la orientación de un punto p con respecto a una línea definida por
    los puntos p1 y p2.
    """
    return (p2[0] - p1[0]) * (p[1] - p1[1]) - (p2[1] - p1[1]) * (p[0] - p1[0])

def distanciaLinea(p, p1, p2):
    """
    Función que calcula la distancia perpendicular de un punto p a la línea definida por los
    puntos p1 y p2.
    """
    return abs((p2[1] - p1[1]) * p[0] - (p2[0] - p1[0]) * p[1] + p2[0] * p1[1] - p2[1] * p1[0])

def encontrarQuickhull(P, p1, p2):
    """
    Función recursiva que encuentra los puntos que forman parte del cierre convexo en un lado
    de la línea p1-p2.
    """
    if not P:
        return []
    mas_lejos = max(P, key=lambda x: distanciaLinea(x, p1, p2))
    P.remove(mas_lejos)
    izq = [p for p in P if orientacionPuntoRecta(p1, mas_lejos, p) > 0]
    der = [p for p in P if orientacionPuntoRecta(mas_lejos, p2, p) > 0]
    return encontrarQuickhull(izq, p1, mas_lejos) + [mas_lejos] + encontrarQuickhull(der, mas_lejos, p2)

def Quickhull(P):
    """
    Función principal que implementa el algoritmo Quickhull para calcular el cierre convexo.
    Divide los puntos en dos mitades (superior e inferior) con respecto a los puntos extremos,
    y luego encuentra los puntos del cierre convexo en cada mitad.
    """
    min_abs = minX(P)
    max_abs = maxX(P)
    superior = [p for p in P if orientacionPuntoRecta(min_abs, max_abs, p) > 0]
    inferior = [p for p in P if orientacionPuntoRecta(max_abs, min_abs, p) > 0]
    return [min_abs] + encontrarQuickhull(superior, min_abs, max_abs) + [max_abs] + encontrarQuickhull(inferior, max_abs, min_abs)

@interact
def pedirPuntos(p=slider(3, 100, 1, 1, label='Número de Puntos')):
    """
    Función interactiva para seleccionar el número de puntos a generar.
    """
    @interact
    def generaPuntos(distribucion=selector([(1, 'Uniforme'), (2, 'Normal')], label='Distribución')):
        """
        Función interactiva para seleccionar la distribución de los puntos (Uniforme o Normal).
        """
        if distribucion == 1:
            P = [[random(), random()] for _ in range(p)]  # Usa random.random()
        else:
            P = [[2 * gauss(0, 1), gauss(0, 1)] for _ in range(p)]
        CH = Quickhull(P)
        @interact
        def visualizarPaso(paso=slider(0, len(CH) - 1, 1, 0, label='Paso')):
            """
            Función interactiva para visualizar el proceso paso a paso.
            Muestra los puntos parciales que forman la envolvente convexa.
            """
            parciales = CH[:paso + 1]
            show(
                list_plot(P, color='blue', size=20, legend_label='Puntos') +
                polygon(parciales + [parciales[0]], color='hotpink', alpha=0.5, legend_label='Envolvente') +
                point(parciales, color='green', size=30, legend_label='Puntos Envolvente')
            )

Interactive function <function pedirPuntos at 0x6ffec43bfb90> with 1 widget
  p: TransformIntSlider(value=3, d…